# CDBS Programming Test Data Prep
Steps
- download 2022 prescription and provider data from:
  - https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug
    - data dictionary: https://data.cms.gov/resources/medicare-part-d-prescribers-by-provider-and-drug-data-dictionary
  - https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider
    - data dictionary: https://data.cms.gov/resources/medicare-part-d-prescribers-by-provider-data-dictionary
- rename datasets
  - MUP_DPR_RY24_P04_V10_DY22_NPIBN.csv -> prescribers_provider_and_drug.csv
  - MUP_DPR_RY24_P04_V10_DY22_NPI.csv -> prescribers_provider.csv
- load data into pandas
- drop shared columns
- randomly select 10,000 providers
- subset datases to the randomly selected providers
- save Rx and provider data in sqlite db
- save drug name data to sqlite db

In [1]:
import pandas as pd
import sqlite3
import random

## load prescription and provider tables

In [2]:
rx_df = pd.read_csv('../data/prescribers_provider_and_drug.csv', encoding='ISO-8859-1', dtype={'Prscrbr_State_FIPS': 'string'})

In [3]:
len(rx_df)

25869521

In [4]:
provider_df = pd.read_csv('../data/prescribers_provider.csv', encoding='ISO-8859-1')

In [5]:
len(provider_df)

1332309

In [6]:
shared_cols = list(set(rx_df.columns).intersection(set(provider_df.columns)))
shared_cols

['GE65_Sprsn_Flag',
 'Prscrbr_State_FIPS',
 'GE65_Tot_Drug_Cst',
 'GE65_Tot_Day_Suply',
 'Tot_Drug_Cst',
 'Prscrbr_First_Name',
 'Tot_30day_Fills',
 'Prscrbr_City',
 'Prscrbr_State_Abrvtn',
 'Prscrbr_NPI',
 'Prscrbr_Type',
 'GE65_Tot_Benes',
 'Tot_Benes',
 'Prscrbr_Last_Org_Name',
 'Prscrbr_Type_Src',
 'GE65_Tot_Clms',
 'GE65_Bene_Sprsn_Flag',
 'Tot_Day_Suply',
 'GE65_Tot_30day_Fills',
 'Tot_Clms']

## drop shared columns from prescriptions for simplicty
note: keep the `Prscrbr_NPI` column

In [7]:
shared_cols.remove('Prscrbr_NPI')
'Prscrbr_NPI' in shared_cols

False

In [8]:
rx_df = rx_df.drop(columns=shared_cols, axis=1)

## randomly choose 10,000 providers

In [9]:
len(provider_df.Prscrbr_NPI.to_list())

1332309

In [10]:
prv_ids = random.sample(provider_df.Prscrbr_NPI.to_list(), 10_000)
len(set(prv_ids))

10000

## subset tables

In [11]:
prv_df_subset = provider_df[provider_df.Prscrbr_NPI.isin(prv_ids)]
prv_df_subset.set_index('Prscrbr_NPI', inplace=True) # for the providers table use Prscrbr_NPI as the index
len(prv_df_subset)

10000

In [12]:
rx_df_subset = rx_df[rx_df.Prscrbr_NPI.isin(prv_ids)]
len(rx_df_subset)

194969

In [13]:
len(list(rx_df_subset.Prscrbr_NPI.unique())) # note: not all 10,000 Prscrbr_NPI were found in provider and drug data subset

7995

In [14]:
len(list(rx_df_subset.Prscrbr_NPI.unique()))

7995

## save prescription and provider data to sqlite

In [15]:
con = sqlite3.connect('../data/drug_info.db')

In [16]:
prv_df_subset.to_sql('provider', con, if_exists='replace')

10000

In [17]:
rx_df_subset.to_sql('prescription', con, if_exists='replace', index=False)

194969

## save drug name data to sqlite

In [18]:
drug_name_df = pd.read_csv('../data/drug-names.csv')
len(drug_name_df)

466

In [19]:
drug_name_df.to_sql('drug_name', con, if_exists='replace', index=False)

466